In [1]:
#intial imports: 
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

import numpy as np
from numpy import asarray
# import pandas as pd
import tensorflow as tf
import os
import csv
# from tensorflow.keras.losses import SparseCategoricalCrossentropy
# from tensorflow.keras.optimizers import Adam
import time
# import streamlit as st
from matplotlib import pyplot as plt
from PIL import Image

2024-01-20 16:20:41.050264: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-20 16:20:41.096678: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-20 16:20:41.098168: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-20 16:20:41.881148: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### setting up the necessary files for data transcription

In [2]:
def setup_file(csv_file):
    rows = []
    #creating empty file in folder, i added the start_time in the name of the csv file, so that if a symbol appears many times in a video, it will still be created in two different csv files, just that they will have different starting times
    # csv_file = f"/users/aly/documents/programming/apps/machine learning/asl converter/training_models/mediapipe/demo_test/demo.csv"
    # csv_file="d:/personnel/other learning/programming/personal_projects/asl_language_translation/training_models/mediapipe/demo_test/demo.csv"
    # if os.path.exists(csv_file):
    #     return 



# Setup CSV File for the videos
# 21 right hand landmarks, 21 left hand landmarks, 33 pose landmarks
    num_coords = 21 + 21 #+ 33
    landmarks = []
    for val in range(1, num_coords+1):
        landmarks += ['x{}'.format(val), 'y{}'.format(val)]#, 'z{}'.format(val), 'v{}'.format(val)]
    print("Initialized an empty landmarks of size:", len(landmarks))

    with open(csv_file, mode='w', newline='') as f:
        csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        csv_writer.writerow(landmarks)
    

### extracting the hand coordinates: 
###### https://arkalsekar.medium.com/how-to-get-all-the-co-ordinates-of-hand-using-mediapipe-hand-solutions-ac7e2742f702
###### https://www.futurelearn.com/info/courses/introduction-to-image-analysis-for-plant-phenotyping/0/steps/305359


In [3]:
def open_cam(csv_file):   
  rows = []
  # For webcam input:
  cap = cv2.VideoCapture(0)
  with mp_hands.Hands(
      min_detection_confidence=0.5,
      min_tracking_confidence=0.5) as hands:
    while cap.isOpened():


      success, image = cap.read()
      if not success:
        print("Ignoring empty camera frame.")
        # If loading a video, use 'break' instead of 'continue'.
        continue

      # Flip the image horizontally for a later selfie-view display, and convert
      # the BGR image to RGB.
      image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
      # To improve performance, optionally mark the image as not writeable to
      # pass by reference.
      image.flags.writeable = False

      #!results contains all the information about the image, in this case, we are looking at hands
      results = hands.process(image)
      
      image_height, image_width, _ = image.shape
      # Draw the hand annotations on the image.
      image.flags.writeable = True
      image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

      cv2.imshow('MediaPipe Hands', image)
      
      #stop the process
      if cv2.waitKey(5) & 0xFF == ord('q'): 
          break
      
      print(type(results.multi_hand_landmarks))
      print('-'*50, '\n', results.multi_hand_landmarks)
      
      # #!checks for both hands, and looks if there is data
      # if results.multi_hand_landmarks:
      #   #!extracting the information from the right hand
      #   for hand in results.multi_hand_landmarks:

      #     both_hand = hand
      #     hand_row = list(np.array([[landmark.x, landmark.y] for ids, landmark in both_hand]).flatten())

      if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
          empty_row = []
          # Here is How to Get All the Coordinates
          for ids, landmrk in enumerate(hand_landmarks.landmark):
              # print(ids, landmrk)
              cx, cy = landmrk.x * image_width, landmrk.y*image_height
              empty_row.append(cx)
              empty_row.append(cy)
              # print(cx, cy)
          print(enumerate(hand_landmarks.landmark))            # print (ids, cx, cy)
          mp_drawing.draw_landmarks(
              image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
          rows.append(empty_row)
        # #!else, I wanna just write 0 for the information about the hands.
        else: 
        #   both_hand_row = list(np.array([[0,0] for i in range(42)]).flatten())
          empty_row = [0 for i in range(42)]
          rows.append(empty_row)


  # After the loop release the cap object 
  cap.release() 

  #closes all instance of the camera
  cv2.destroyAllWindows()

  with open(csv_file, mode='a', newline='') as f:
                        csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                        for row in rows:
                            csv_writer.writerow(row) 

In [4]:
optional_int = 1
csv_file = f"data/csv_file{optional_int}.csv"
setup_file(csv_file)
open_cam(csv_file)

Initialized an empty landmarks of size: 84


libEGL warning: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object fi

<class 'NoneType'>
-------------------------------------------------- 
 None
<class 'NoneType'>
-------------------------------------------------- 
 None
<class 'NoneType'>
-------------------------------------------------- 
 None
<class 'NoneType'>
-------------------------------------------------- 
 None
<class 'NoneType'>
-------------------------------------------------- 
 None
<class 'NoneType'>
-------------------------------------------------- 
 None
<class 'NoneType'>
-------------------------------------------------- 
 None
<class 'NoneType'>
-------------------------------------------------- 
 None
<class 'NoneType'>
-------------------------------------------------- 
 None
<class 'NoneType'>
-------------------------------------------------- 
 None
<class 'NoneType'>
-------------------------------------------------- 
 None
<class 'NoneType'>
-------------------------------------------------- 
 None
<class 'NoneType'>
-------------------------------------------------- 
 None

TypeError: list.append() takes exactly one argument (2 given)

: 

In [ ]:
# # import the opencv library 
# import cv2 
  
  
# # define a video capture object 
# vid = cv2.VideoCapture(0) 
  
# while(True): 
      
#     # Capture the video frame 
#     # by frame 
#     ret, frame = vid.read() 
  
#     # Display the resulting frame 
#     cv2.imshow('frame', frame) 

#     #extracting the frames  
    
    

#     # the 'q' button is set as the 
#     # quitting button you may use any 
#     # desired button of your choice 
#     if cv2.waitKey(1) & 0xFF == ord('q'): 
#         break
  
# # After the loop release the cap object 
# vid.release() 
# # Destroy all the windows 
# cv2.destroyAllWindows() 